In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from tensorflow.keras.models import load_model

In [2]:
detector = cv2.CascadeClassifier("haar_carplate.xml")
cnn = load_model('cnn_model_car.h5')

mnist_label_name = [i for i in range(10)]
AZ_label_name = [chr(i+65) for i in range(26)]
label_name = np.hstack([mnist_label_name, AZ_label_name])

In [ ]:
VIDEO_IN = cv2.VideoCapture(0)
while True:
    hasFrame, img = VIDEO_IN.read()
    cv2.imshow("Frame", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
VIDEO_IN.release()
cv2.destroyAllWindows()

In [3]:
VIDEO_IN = cv2.VideoCapture('carvideo.mp4')

while True:
    hasFrame, img = VIDEO_IN.read()
    img = cv2.resize(img, None, fx=1.4, fy=1.4)
    signs = detector.detectMultiScale(img, minSize = (76, 20), scaleFactor = 1.1, minNeighbors=8)
    if len(signs) > 0:
        for (sx, sy, sw, sh) in signs:
            crop_img = img[sy:sy+sh, sx:sx+sw]
            gray_img = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
            _, binary_img = cv2.threshold(gray_img, 127, 255, cv2.THRESH_BINARY_INV)
                              
            save_predict_name = []
            save_contours = []
            contours, hierarchy = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            for i in range(len(contours)):
                (x, y, w, h) = cv2.boundingRect(contours[i])
                save_contours.append((x, y, w, h))
            save_contours = sorted(save_contours, key=lambda x:x[0])
        
            for (x, y, w, h) in save_contours:
                if sw*(3/20)>w>sw*(0.5/20) and sh*(19.5/20)>h>sh*(12/20):
                    number = gray_img[y:y+h, x:x+w]
                    re_number = cv2.resize(number, (28, 28), interpolation=cv2.INTER_CUBIC)
                    intput_number = np.expand_dims(np.expand_dims(re_number.astype('float32')/255,-1),0)
                    predict = cnn.predict(intput_number)
                    predict_name = label_name[np.argmax(predict)]
                    save_predict_name.append(predict_name)
            
            if len(save_predict_name)>5:
                cv2.rectangle(img, (sx, sy), (sx+sw, sy+sh), (0, 0, 255), 2)
                cv2.putText(img, "".join(save_predict_name), (sx, int(sy-sh/4)), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
            else:
                cv2.putText(img, "Not clear", (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    else:
        cv2.putText(img, "No License_Plate", (20, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 2)
    cv2.imshow("Frame", img)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
VIDEO_IN.release()
cv2.destroyAllWindows()